У цьому ДЗ ми потренуємось розв'язувати задачу багатокласової класифікації за допомогою логістичної регресії з використанням стратегій One-vs-Rest та One-vs-One, оцінити якість моделей та порівняти стратегії.

In [56]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.metrics import accuracy_score, confusion_matrix,f1_score,roc_curve, auc, roc_auc_score,root_mean_squared_error, r2_score
from sklearn.metrics import classification_report, precision_score, recall_score
from imblearn.over_sampling import SMOTENC
from imblearn.combine import SMOTETomek
from sklearn.pipeline import Pipeline

### Опис задачі і даних

**Контекст**

В цьому ДЗ ми працюємо з даними про сегментацію клієнтів.

Сегментація клієнтів – це практика поділу бази клієнтів на групи індивідів, які схожі між собою за певними критеріями, що мають значення для маркетингу, такими як вік, стать, інтереси та звички у витратах.

Компанії, які використовують сегментацію клієнтів, виходять з того, що кожен клієнт є унікальним і що їхні маркетингові зусилля будуть більш ефективними, якщо вони орієнтуватимуться на конкретні, менші групи зі зверненнями, які ці споживачі вважатимуть доречними та які спонукатимуть їх до купівлі. Компанії також сподіваються отримати глибше розуміння уподобань та потреб своїх клієнтів з метою виявлення того, що кожен сегмент цінує найбільше, щоб точніше адаптувати маркетингові матеріали до цього сегменту.

**Зміст**.

Автомобільна компанія планує вийти на нові ринки зі своїми існуючими продуктами (P1, P2, P3, P4 і P5). Після інтенсивного маркетингового дослідження вони дійшли висновку, що поведінка нового ринку схожа на їхній існуючий ринок.

На своєму існуючому ринку команда з продажу класифікувала всіх клієнтів на 4 сегменти (A, B, C, D). Потім вони здійснювали сегментовані звернення та комунікацію з різними сегментами клієнтів. Ця стратегія працювала для них надзвичайно добре. Вони планують використати ту саму стратегію на нових ринках і визначили 2627 нових потенційних клієнтів.

Ви маєте допомогти менеджеру передбачити правильну групу для нових клієнтів.

В цьому ДЗ використовуємо дані `customer_segmentation_train.csv`[скачати дані](https://drive.google.com/file/d/1VU1y2EwaHkVfr5RZ1U4MPWjeflAusK3w/view?usp=sharing). Це `train.csv`з цього [змагання](https://www.kaggle.com/datasets/abisheksudarshan/customer-segmentation/data?select=train.csv)

**Завдання 1.** Завантажте та підготуйте датасет до аналізу. Виконайте обробку пропущених значень та необхідне кодування категоріальних ознак. Розбийте на тренувальну і тестувальну вибірку, де в тесті 20%. Памʼятаємо, що весь препроцесинг ліпше все ж тренувати на тренувальній вибірці і на тестувальній лише використовувати вже натреновані трансформери.
Але в даному випадку оскільки значень в категоріях небагато, можна зробити обробку і на оригінальних даних, а потім розбити - це простіше. Можна також реалізувати процесинг і тренування моделі з пайплайнами. Обирайте як вам зручніше.

In [4]:
raw_df = pd.read_csv('customer_segmentation_train.csv')
raw_df.head()

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1,Segmentation
0,462809,Male,No,22,No,Healthcare,1.0,Low,4.0,Cat_4,D
1,462643,Female,Yes,38,Yes,Engineer,NaN,Average,3.0,Cat_4,A
2,466315,Female,Yes,67,Yes,Engineer,1.0,Low,1.0,Cat_6,B
3,461735,Male,Yes,67,Yes,Lawyer,0.0,High,2.0,Cat_6,B
4,462669,Female,Yes,40,Yes,Entertainment,NaN,High,6.0,Cat_6,A


In [8]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   ID               8068 non-null   int64  
 1   Gender           8068 non-null   object 
 2   Ever_Married     7928 non-null   object 
 3   Age              8068 non-null   int64  
 4   Graduated        7990 non-null   object 
 5   Profession       7944 non-null   object 
 6   Work_Experience  7239 non-null   float64
 7   Spending_Score   8068 non-null   object 
 8   Family_Size      7733 non-null   float64
 9   Var_1            7992 non-null   object 
 10  Segmentation     8068 non-null   object 
dtypes: float64(2), int64(2), object(7)
memory usage: 693.5+ KB


In [18]:
raw_df.Profession.unique()

array(['Healthcare', 'Engineer', 'Lawyer', 'Entertainment', 'Artist',
       'Executive', 'Doctor', 'Homemaker', 'Marketing', nan], dtype=object)

In [66]:
X = raw_df.drop(columns=['Segmentation'])
y = raw_df['Segmentation']

In [67]:
numeric_cols = X.select_dtypes(include=np.number).columns.tolist()
categorical_cols = X.select_dtypes('object').columns.tolist()

In [68]:
num_imputer = SimpleImputer(strategy='mean')
X[numeric_cols] = num_imputer.fit_transform(X[numeric_cols])

cat_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_cols] = cat_imputer.fit_transform(X[categorical_cols])

# encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
# X_encoded = pd.DataFrame(
#     encoder.fit_transform(X[categorical_cols]),
#     columns=encoder.get_feature_names_out(categorical_cols),
#     index=X.index
# )

# X_final = pd.concat([X[numeric_cols], X_encoded], axis=1)

In [59]:
# X_final.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8068 entries, 0 to 8067
Data columns (total 29 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ID                        8068 non-null   float64
 1   Age                       8068 non-null   float64
 2   Work_Experience           8068 non-null   float64
 3   Family_Size               8068 non-null   float64
 4   Gender_Female             8068 non-null   float64
 5   Gender_Male               8068 non-null   float64
 6   Ever_Married_No           8068 non-null   float64
 7   Ever_Married_Yes          8068 non-null   float64
 8   Graduated_No              8068 non-null   float64
 9   Graduated_Yes             8068 non-null   float64
 10  Profession_Artist         8068 non-null   float64
 11  Profession_Doctor         8068 non-null   float64
 12  Profession_Engineer       8068 non-null   float64
 13  Profession_Entertainment  8068 non-null   float64
 14  Professi

In [69]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,stratify=y)

**Завдання 2. Важливо уважно прочитати все формулювання цього завдання до кінця!**

Застосуйте методи ресемплингу даних SMOTE та SMOTE-Tomek з бібліотеки imbalanced-learn до тренувальної вибірки. В результаті у Вас має вийти 2 тренувальних набори: з апсемплингом зі SMOTE, та з ресамплингом з SMOTE-Tomek.

Увага! В нашому наборі даних є як категоріальні дані, так і звичайні числові. Базовий SMOTE не буде правильно працювати з категоріальними даними, але є його модифікація, яка буде. Тому в цього завдання є 2 виконання

  1. Застосувати SMOTE базовий лише на НЕкатегоріальних ознаках.

  2. Переглянути інформацію про метод [SMOTENC](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTENC.html#imblearn.over_sampling.SMOTENC) і використати цей метод в цій задачі. За цей спосіб буде +3 бали за це завдання і він рекомендований для виконання.

  **Підказка**: аби скористатись SMOTENC треба створити змінну, яка містить індекси ознак, які є категоріальними (їх номер серед колонок) і передати при ініціації екземпляра класу `SMOTENC(..., categorical_features=cat_feature_indeces)`.
  
  Ви також можете розглянути варіант використання варіації SMOTE, який працює ЛИШЕ з категоріальними ознаками [SMOTEN](https://imbalanced-learn.org/dev/references/generated/imblearn.over_sampling.SMOTEN.html)

In [41]:
pip install imbalanced-learn

In [70]:
categorical_indices = [X_train.columns.get_loc(c) for c in categorical_cols]

In [71]:
smotenc = SMOTENC(categorical_features=categorical_indices, random_state=42)
X_train_smotenc, y_train_smotenc = smotenc.fit_resample(X_train, y_train)

print("Original training set size:", y_train.value_counts().to_dict())
print("After SMOTENC:", y_train_smotenc.value_counts().to_dict())

Original training set size: {'D': 1814, 'A': 1578, 'C': 1576, 'B': 1486}
After SMOTENC: {'A': 1814, 'B': 1814, 'C': 1814, 'D': 1814}


In [73]:
X_train_smotenc

,ID,Gender,Ever_Married,Age,Graduated,Profession,Work_Experience,Spending_Score,Family_Size,Var_1
0,465905.000000,Female,No,32.000000,Yes,Artist,9.000000,Low,1.000000,Cat_6
1,462903.000000,Male,Yes,72.000000,Yes,Entertainment,2.641663,Average,2.000000,Cat_6
2,467901.000000,Female,No,33.000000,Yes,Entertainment,1.000000,Low,4.000000,Cat_6
3,463613.000000,Female,Yes,48.000000,Yes,Artist,0.000000,Average,6.000000,Cat_6
4,459859.000000,Female,Yes,28.000000,No,Doctor,9.000000,Low,1.000000,Cat_7
...,...,...,...,...,...,...,...,...,...,...
7251,459113.314935,Male,Yes,71.820691,Yes,Artist,0.940230,High,2.799311,Cat_6
7252,461894.669784,Female,Yes,38.632113,Yes,Artist,2.122629,Average,2.849052,Cat_6
7253,463145.033268,Male,Yes,42.341961,Yes,Artist,3.717189,Average,3.223660,Cat_6
7254,466599.783559,Female,Yes,43.573153,Yes,Artist,6.146306,Average,2.000000,Cat_6


In [98]:
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
X_encoded = pd.DataFrame(
    encoder.fit_transform(X_train[categorical_cols]),
    columns=encoder.get_feature_names_out(categorical_cols),
    index=X_train.index
)

X_train_encoded = pd.concat([X_train[numeric_cols], X_encoded], axis=1)

In [99]:
X_train_encoded.shape, y_train.shape

((6454, 29), (6454,))

In [87]:
from imblearn.combine import SMOTETomek

smote_tomek = SMOTETomek(random_state=42)
X_train_smotetomek, y_train_smotetomek = smote_tomek.fit_resample(X_train_encoded, y_train)


In [88]:
print("Original training set size:", y_train.value_counts().to_dict())
print("After SMOTE-Tomek:", y_train_smotetomek.value_counts().to_dict())

Original training set size: {'D': 1814, 'A': 1578, 'C': 1576, 'B': 1486}
After SMOTE-Tomek: {'C': 1393, 'D': 1374, 'B': 1310, 'A': 1269}


**Завдання 3**.
  1. Навчіть модель логістичної регресії з використанням стратегії One-vs-Rest з логістичною регресією на оригінальних даних, збалансованих з SMOTE, збалансованих з Smote-Tomek.  
  2. Виміряйте якість кожної з натренованих моделей використовуючи `sklearn.metrics.classification_report`.
  3. Напишіть, яку метрику ви обрали для порівняння моделей.
  4. Яка модель найкраща?
  5. Якщо немає суттєвої різниці між моделями - напишіть свою гіпотезу, чому?

#### one-vs-rest

In [89]:
from sklearn.multiclass import OneVsRestClassifier, OneVsOneClassifier

In [100]:
X_encoded_t = pd.DataFrame(
    encoder.fit_transform(X_test[categorical_cols]),
    columns=encoder.get_feature_names_out(categorical_cols),
    index=X_test.index
)

X_test_encoded = pd.concat([X_test[numeric_cols], X_encoded_t], axis=1)


In [94]:
log_reg = LogisticRegression(solver='liblinear')
ovr_model = OneVsRestClassifier(log_reg)
ovr_model.fit(X_train_encoded, y_train)
ovr_predictions = ovr_model.predict(X_test_encoded)

print(classification_report(y_test, ovr_predictions))

              precision    recall  f1-score   support

           A       0.00      0.00      0.00       394
           B       0.00      0.00      0.00       372
           C       0.00      0.00      0.00       394
           D       0.28      1.00      0.44       454

    accuracy                           0.28      1614
   macro avg       0.07      0.25      0.11      1614
weighted avg       0.08      0.28      0.12      1614



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### smotenc

In [105]:
X_encoded_s = pd.DataFrame(
    encoder.fit_transform(X_train_smotenc[categorical_cols]),
    columns=encoder.get_feature_names_out(categorical_cols),
    index=X_train_smotenc.index
)

X_train_encoded_sm = pd.concat([X_train_smotenc[numeric_cols], X_encoded_s], axis=1)

In [106]:
ovr_model.fit(X_train_encoded_sm, y_train_smotenc)
ovr_predictions_smotenc = ovr_model.predict(X_test_encoded)

print(classification_report(y_test, ovr_predictions_smotenc))

              precision    recall  f1-score   support

           A       0.00      0.00      0.00       394
           B       0.00      0.00      0.00       372
           C       0.24      1.00      0.39       394
           D       0.00      0.00      0.00       454

    accuracy                           0.24      1614
   macro avg       0.06      0.25      0.10      1614
weighted avg       0.06      0.24      0.10      1614



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


#### smote-tomek

In [107]:
ovr_model.fit(X_train_smotetomek, y_train_smotetomek)
ovr_predictions_smotetomek = ovr_model.predict(X_test_encoded)

print(classification_report(y_test, ovr_predictions_smotetomek))

              precision    recall  f1-score   support

           A       0.00      0.00      0.00       394
           B       0.00      0.00      0.00       372
           C       0.32      0.70      0.44       394
           D       0.47      0.79      0.59       454

    accuracy                           0.39      1614
   macro avg       0.20      0.37      0.26      1614
weighted avg       0.21      0.39      0.27      1614



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Якщо порівнювати модель по avarage f1-score, то видно що вони впринципі всі не дуже))

але SMOTE-Tomek показав себе найкраще, що я і очікувала, бо це комбінація двох методів, і ця модель як мінімум розрізняє 2 класи.

Взагалі, так розумію, що для цих даних логістична регресія поганий вибір, тому такі не дуже добрі результати